In [1]:
# Import appropriate modules from the client library.
from googleads import dfp
from collections import defaultdict
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
import pandas as pd
import time
sc.stop()

sc =SparkContext()
sqlCtx = SQLContext(sc)


In [2]:
orders = sqlCtx.read.parquet("gs://ds-url-catag/dfp_new/orders_final/")

In [7]:
from pyspark.sql.types import BooleanType,StringType
import re

def regex_filter(x):
    list_campaign = ['DBS','Gojek','indosat','grab','Google roadblock','uber','opera']
    filter_status = ''
    string_x = x[:5]
    for camp in list_campaign:
        if camp.lower() in string_x.lower():
            filter_status = True
    
    return filter_status
    
    
filter_udf = udf(regex_filter, BooleanType())

orders = orders.withColumn('Filter_campaigns',filter_udf(orders.Order_name))

In [11]:
new_campaigns = orders.where(col('Filter_campaigns') == True).dropDuplicates()

In [24]:
new_campaigns.write.mode('overwrite').parquet("gs://ds-url-catag/campaign_analysis/campaign_details/")